In [ ]:
import pandas as pd
import numpy as np

# Task 0
Data extraction: get the data from 3 tables & combine it into single `.csv` file.
After that read this file using pandas to create Dataframe.
So it will be all joined data in 1 dataframe. Quick check - should be 74818 rows in it.

In [ ]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt

# 1. Connect and Extract
conn = sqlite3.connect('restaurant_data.db')

query = """
SELECT 
    oi.*, 
    o.order_date, 
    p.product_name, 
    p.price as unit_price
FROM order_items oi
LEFT JOIN orders o ON oi.order_id = o.id
LEFT JOIN products p ON oi.product_id = p.id
"""

# 2. Save to CSV
raw_data = pd.read_sql_query(query, conn)
raw_data.to_csv('combined_restaurant_data.csv', index=False)
conn.close()

# 3. Load into DataFrame
df = pd.read_csv('combined_restaurant_data.csv')

# Quick check: should be 74818 rows
print(f"Total rows: {len(df)}")

# Task 1
Get Top 10 most popular products in restaurant sold by Quantity.
Count how many times each product was sold and create a pie chart with percentage of popularity (by quantity) for top 10 of them.

Example:

![pie chart](../demo/pie.png)

In [ ]:
# Grouping by quantity
top_qty = df.groupby('product_name')['quantity'].sum().sort_values(ascending=False).head(10)

# Plotting
plt.figure(figsize=(10, 7))
top_qty.plot(kind='pie', autopct='%1.1f%%', startangle=140, cmap='viridis')
plt.title('Top 10 Products by Quantity Sold')
plt.ylabel('') # Hides the 'quantity' label for aesthetics
plt.show()

# Task 2
Calculate `Item Price` (Product Price * Quantity) for each Order Item in dataframe.
And Make the same Top 10 pie chart, but this time by `Item Price`. So this chart should describe not the most popular products by quantity, but which products (top 10) make the most money for restaurant. It should be also with percentage.

In [ ]:
# Calculate Item Price
df['item_price'] = df['unit_price'] * df['quantity']

# Grouping by Item Price (Revenue)
top_revenue = df.groupby('product_name')['item_price'].sum().sort_values(ascending=False).head(10)

# Plotting
plt.figure(figsize=(10, 7))
top_revenue.plot(kind='pie', autopct='%1.1f%%', startangle=140, cmap='magma')
plt.title('Top 10 Products by Revenue Contribution')
plt.ylabel('')
plt.show()

# Task 3
Calculate `Order Hour` based on `Order Datetime`, which will tell about the specific our the order was created (from 0 to 23). Using `Order Hour` create a bar chart, which will tell the total restaurant income based on the hour order was created. So on x-axis - it will be values from 0 to 23 (hours), on y-axis - it will be the total sum of order prices, which were sold on that hour.

Example:

![bar chart](../demo/bar.png)

In [ ]:
# Convert to datetime and extract the hour
df['order_date'] = pd.to_datetime(df['order_date'])
df['order_hour'] = df['order_date'].dt.hour

# Aggregate income by hour
hourly_income = df.groupby('order_hour')['item_price'].sum()

# Plotting
plt.figure(figsize=(12, 6))
hourly_income.plot(kind='bar', color='skyblue', edgecolor='black')
plt.title('Total Restaurant Income by Hour')
plt.xlabel('Hour of Day (0-23)')
plt.ylabel('Total Revenue')
plt.xticks(rotation=0)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

# Task 4
Make similar bar chart, but right now with `Order Day Of The Week` (from Monday to Sunday), and also analyze total restaurant income by each day of the week.

In [ ]:
# 1. Extract the day of the week
# We use .dt.day_name() to get 'Monday', 'Tuesday', etc.
df['day_of_week'] = df['order_date'].dt.day_name()

# 2. Define the correct categorical order for sorting the chart
days_order = [
    'Monday', 'Tuesday', 'Wednesday', 'Thursday', 
    'Friday', 'Saturday', 'Sunday'
]

# 3. Group by day and calculate total income
daily_income = df.groupby('day_of_week')['item_price'].sum().reindex(days_order)

# 4. Visualization
plt.figure(figsize=(12, 6))
daily_income.plot(kind='bar', color='teal', edgecolor='black')

plt.title('Total Restaurant Income by Day of the Week', fontsize=14)
plt.xlabel('Day of the Week', fontsize=12)
plt.ylabel('Total Revenue', fontsize=12)
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.6)

plt.tight_layout()
plt.show()